In [3]:
k  = "2.json" # ONLY CHANGE THIS
fh = open(k,'r')
line = fh.readline()
import json
inp = json.loads(line)
lis = inp['vstrands']
fn = 'kinetic trap detector'
f = open(fn,'w')
kt = 0   #if this value is still 0 by the end of the program, that means no kinetic trap is detected

#this search function searches for all crossover pairs within a strand, and outputs the coordinates of these pairs,
#as well as the helice number they cross into, as 2 separate lists
def search(x,l,n,t): 
    scafpairs = []  #if search func is being fed a scaffold list, it will output the list of crossover pairs as 'scafpairs'
    stappairs = []  #otherwise if it's being fed a staple list, it will output the list of crossover pairs as 'stappairs'
    crossB = []
    crossD = []        #crossB is for indices a and b, crossD is for indices c and d
    if True:
        for k in range(x):                           #cross holds the information of coordinates at which crossovers occur
            if l[k][0] != -1 and l[k][0] != n:       #l[k][0] (index a) and l[k][2] (index c) should normally be equal to n; if not, then either a  
                crossB.append(l[k])                  #crossover occured (in which case it's equal to the helice number 
            if l[k][2] != -1 and l[k][2] != n:       #it crossed over into) or it's an empty position, in which case it's -1.
                crossD.append(l[k])        #if index a satisfied the crossover condition above, call it a 'a' crossover, and likewise for a 'c' crossover
    B = []  #this will contain all the horizontal coordinates of 'a' crossovers
    D = []  #this will contain all the horizontal coordinates of 'c' crossovers
    for m in range(len(crossB)):
        if crossB[m][1] != -1:       #makes sure that the coordinate is not empty; just because index a is not empty as
            B.append(crossB[m][1])   #checked above does not necessarily mean that index b is also not empty
    for m in range(len(crossD)):
        if crossD[m][3] != -1:
            D.append(crossD[m][3])
    sortB = list(set(sorted(B)))  #sorted() sorts a list of numbers in increasing order
    sortD = list(set(sorted(D)))  #set() removes any repeating coordinates, so we don't get repeated output
    Binto = []   #these lists will hold the strand number that every crossover pair crosses into. The index of this list
    Dinto = []   #complements the index of sortB/sortD (i.e. the nth index of sortB corresponds to the nth index of Binto)
    for r in range(len(sortB)):      #note that this also implies that len(sortB) == len(Binto) == len(sortD) == len(Dinto)
        for s in range(len(crossB)):
            if sortB[r] == crossB[s][1]:
                Binto.append(crossB[s][0])
    for r in range(len(sortD)):
        for s in range(len(crossD)):
            if sortD[r] == crossD[s][3]:
                Dinto.append(crossD[s][2])  
    temp = []     #temporary list, used for appending
    pairs = []    #this will hold the coordinates of crossover pairs
    into = []     #this will contain the strand numbers corresponding to the list of coordinate pairs
    for e in range(len(sortB)):
        for f in range(len(sortD)):
            if abs(sortB[e] - sortD[f]) == 1 and Binto[e] == Dinto[f]:
                temp = list(set(sorted([sortB[e],sortD[f]])))
                pairs.append(temp)
                temp = []
                into.append(Binto[e])
    if t == 'scaf':
        scafpairs.append(pairs)
        return scafpairs, into
    elif t == 'stap':
        stappairs.append(pairs)
        return stappairs, into
          
for i in range(len(lis)):   #loops over all strands. For a given strand i:
    lisA = lis[i]['scaf']   #lisA contains all the lists of indices of 'scaf' for strand i
    A = len(lisA)
    lisB = lis[i]['stap']   #lisB contains all the lists of indices of 'stap' for strand i
    B = len(lisB)
    scafpairs, Into = search(A,lisA,lis[i]['num'],'scaf')   #lis[i]['num'] refers to the principal helice number
    stappairs, into = search(B,lisB,lis[i]['num'],'stap')
    if type(stappairs) == list:  
        if len(stappairs) != 0:   #these weird conditions are necessary because empty output and empty lists sometimes occur
            if len(stappairs[0]) > 1:  
                for u in range(len(stappairs[0])):  #checks the set of all staple crossover pairs against itself to find kinetic traps
                    for v in range(u,len(stappairs[0])):  
                        if into[u] == into[v] and u != v and (abs(stappairs[0][u][1]-stappairs[0][v][0]) <= 5 or abs(stappairs[0][u][0]-stappairs[0][v][1]) <= 5):
                            f.write('a kinetic trap may occur at helice {}, between positions {}-{}, between two staples'.format(i,stappairs[0][u],stappairs[0][v]))
                            f.write('\n')
                            kt = 1  #makes sure that kt != 0, recall that kt = 0 means that no kinetic trap was detected
    if type(scafpairs) == list and type(stappairs) == list:          
        if len(stappairs) != 0 and len(scafpairs) != 0:
            for v in range(len(scafpairs[0])):    #checks the set of scaffold crossover pairs against staple pairs to find kinetic traps
                for u in range(len(stappairs[0])):
                    if into[u] == Into[v] and (abs(stappairs[0][u][1]-scafpairs[0][v][0]) <= 5 or abs(stappairs[0][u][0]-scafpairs[0][v][1]) <= 5):
                        f.write('a kinetic trap may occur at helice {}, between positions {}-{}, between a staple and scaffold'.format(i,stappairs[0][u],scafpairs[0][v]))
                        f.write('\n')
                        kt = 1
if kt == 0:
    f.write('no kinetic trap detected')
fh.close()
f.close()            

FileNotFoundError: [Errno 2] No such file or directory: '2.json'